In [53]:
import numpy as np, pandas as pd
import json
import ast 
from textblob import TextBlob
import nltk
import torch
import pickle
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [54]:
# !conda update pandas --y

In [71]:
# comment out to avoid overwriting original data
# load train data
train = pd.read_csv("data/train_wiki.csv")

In [72]:
train.shape

(100, 6)

### Loading Embedding dictionary

In [73]:
with open("data/dict_embeddings3.pickle", "rb") as f:
    d3 = pickle.load(f)

with open("data/dict_embeddings4.pickle", "rb") as f:
    d4 = pickle.load(f)

In [74]:
# key - sentence / question , val - vector
dict_emb = dict(d3)
dict_emb.update(d4)

In [75]:
len(dict_emb)

7350

In [76]:
del d3, d4

## Data Processing

In [77]:
# find the index of the sentence that contains answer
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [78]:
train

,context,question,answer_start,text,wiki_context,search
0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,515,Saint Bernadette Soubirous,"Our Lady of Lourdes is a Roman Catholic title of the Blessed Virgin Mary venerated in honour of the Marian apparitions that reportedly occurred in 1858 in the vicinity of Lourdes in France. The first of these is the apparition of 11 February 1858, when 14-year old Bernadette Soubirous told her mother that a ""lady"" spoke to her in the cave of Massabielle (a kilometre and a half (1 mi) from the town) while she was gathering firewood with her sister and a friend. Similar apparitions of the ""Lady"" were reported on seventeen occasions that year, until the climax revelation of Our Lady of the Immaculate Conception took place.In 18 January 1862, Pope Pius IX authorized Bishop Bertrand-Sévère Laurence to permit the veneration of the Blessed Virgin Mary in Lourdes. On 3 July 1876, the same Pontiff officially granted a Canonical Coronation to the image that used to be in the courtyard of what is now part of the Rosary Basilica. The image of Our Lady of Lourdes has been widely copied and reproduced in shrines and homes, often in garden landscapes. Soubirous was later canonized as a Catholic saint.",Virgin Mary Lourdes France
1,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"University of Notre Dame's Main Administration Building (known as the Main Building or the ""Golden Dome"") houses various administrative offices, including the Office of the President. Atop of the building stands the Golden Dome, the most recognizable landmark of the University. Three buildings were built at the site; the first was built in 1843 and replaced with a larger one in 1865, which burned down in 1879, after which the third and current building was erected. The building hosts the administrative offices of the University, as well as classrooms, art collections, and exhibition spaces. The building is listed on the national register of historic places.",front Notre Dame Main Building
2,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to

In [63]:
train.shape

(100, 6)

In [64]:
train.dropna(inplace=True)

In [65]:
train.shape

(100, 6)

In [83]:
dict_emb['Architecturally, the school has a Catholic character.']

KeyError: 'Architecturally, the school has a Catholic character.'

In [84]:
def process_data(train):
    
    print("step 1")
    train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    train['wiki_sentences'] = train['wiki_context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    train["target"] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['wiki_sent_emb'] = train['wiki_sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train   

In [85]:
train = process_data(train)

step 1
step 2
step 3
step 4


In [90]:
train

,context,question,answer_start,text,wiki_context,search,sentences,wiki_sentences,target,wiki_sent_emb,quest_emb
0,Archit...,To who...,515,Saint ...,Our La...,Virgin...,[Archi...,[Our L...,5,[[0.10...,[[0.11...
1,Archit...,What i...,188,a copp...,Univer...,front ...,[Archi...,[Unive...,2,"[[0.0,...",[[0.10...
2,Archit...,The Ba...,279,the Ma...,The Ba...,Basili...,[Archi...,[The B...,3,"[[0.0,...",[[0.01...
3,Archit...,What i...,381,a Mari...,The Gr...,Grotto...,[Archi...,[The G...,4,"[[0.0,...",[[0.07...
4,Archit...,What s...,92,a gold...,Univer...,top Ma...,[Archi...,[Unive...,1,"[[0.0,...",[[0.16...
...,...,...,...,...,...,...,...,...,...,...,...
95,In 191...,In 191...,8,Father...,The Un...,presid...,[In 19...,[The U...,0,"[[0.0,...",[[0.03...
96,In 191...,Over h...,66,three ...,The Un...,years ...,[In 19...,[The U...,0,"[[0.0,...",[[0.05...
97,In 191...,Those ...,430,Harvar...,St. Jo...,Jesuit...,[In 19...,[St. J...,2,"[[0.0,...",[[0.02...
98,One of...,The No...,117,Knute ...,This i...,Notre ...,[One o...,[This ...,4,"[[0.0,...",[[0.04...


In [92]:
train.iloc[0,:]['quest_emb']

array([[ 0.11010078,  0.1142294 ,  0.11560896, ...,  0.02811733,
        -0.01866924,  0.14566438]], dtype=float32)

## Predicted Cosine & Euclidean & other Index

In [ ]:
def cosine_sim_wiki(x):
    li = []
    for item in x['wiki_sent_emb']:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li   

In [ ]:
def euc_dist(x,col):
    li = []
    for item in x[col]:
#         print(item)
#         print(x["quest_emb"][0])
#         print()
        li.append(spatial.distance.euclidean(item,x["quest_emb"][0]))
    return li   

In [ ]:
# actually manhatten distance as p = 1
def min_dist(x,col,p=1):
    li = []
    for item in x[col]:
#         print(item)
#         print(x["quest_emb"][0])
#         print()
        li.append(spatial.distance.minkowski(item,x["quest_emb"][0],p))
    return li  

In [ ]:
def che_dist(x,col):
    li = []
    for item in x[col]:
#         print(item)
#         print(x["quest_emb"][0])
#         print()
        li.append(spatial.distance.chebyshev(item,x["quest_emb"][0]))
    return li  

In [93]:
def pred_idx(distances):
    return np.argmin(distances)   

def get_sent_by_idx(x):
    return x['wiki_sentences'][x["pred_idx_cos"]]

def predictions(train):
    
    train["cosine_sim_wiki"] = train.apply(cosine_sim_wiki, axis = 1)
#     train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
#     train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
#     train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
#     train["euclidean_dis2"]= train.apply(euc_dist,axis = 1)
#     train["minkowski_dis"] = train.apply(min_dist,axis = 1)
#     train["chebyshev_dis"] = train.apply(che_dist,axis = 1)
#     del train["diff"]
    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim_wiki"].apply(lambda x: pred_idx(x))
#     train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
#     train["pred_idx_min"] = train["minkowski_dis"].apply(lambda x: pred_idx(x))
#     train["pred_idx_che"] = train["chebyshev_dis"].apply(lambda x: pred_idx(x))
    train["pred_sen_cos"] = train.apply(get_sent_by_idx,axis = 1)


    return train
    

In [94]:
predicted = predictions(train)

cosine start


In [95]:
predicted.head()

,context,question,answer_start,text,wiki_context,search,sentences,wiki_sentences,target,wiki_sent_emb,quest_emb,cosine_sim_wiki,pred_idx_cos,pred_sen_cos
0,Archit...,To who...,515,Saint ...,Our La...,Virgin...,[Archi...,[Our L...,5,[[0.10...,[[0.11...,[0.158...,0,Our La...
1,Archit...,What i...,188,a copp...,Univer...,front ...,[Archi...,[Unive...,2,"[[0.0,...",[[0.10...,"[nan, ...",0,Univer...
2,Archit...,The Ba...,279,the Ma...,The Ba...,Basili...,[Archi...,[The B...,3,"[[0.0,...",[[0.01...,"[nan, ...",0,The Ba...
3,Archit...,What i...,381,a Mari...,The Gr...,Grotto...,[Archi...,[The G...,4,"[[0.0,...",[[0.07...,[nan],0,The Gr...
4,Archit...,What s...,92,a gold...,Univer...,top Ma...,[Archi...,[Unive...,1,"[[0.0,...",[[0.16...,"[nan, ...",0,Univer...


In [ ]:
#2,25,83,98

In [96]:
predicted.iloc[83,:]

context            The un...
question           What i...
answer_start              73
text               Congre...
wiki_context       The Co...
                     ...    
wiki_sent_emb      [[0.0,...
quest_emb          [[0.09...
cosine_sim_wiki    [nan, ...
pred_idx_cos               0
pred_sen_cos       The Co...
Name: 83, Length: 14, dtype: object

In [97]:
predicted

,context,question,answer_start,text,wiki_context,search,sentences,wiki_sentences,target,wiki_sent_emb,quest_emb,cosine_sim_wiki,pred_idx_cos,pred_sen_cos
0,Archit...,To who...,515,Saint ...,Our La...,Virgin...,[Archi...,[Our L...,5,[[0.10...,[[0.11...,[0.158...,0,Our La...
1,Archit...,What i...,188,a copp...,Univer...,front ...,[Archi...,[Unive...,2,"[[0.0,...",[[0.10...,"[nan, ...",0,Univer...
2,Archit...,The Ba...,279,the Ma...,The Ba...,Basili...,[Archi...,[The B...,3,"[[0.0,...",[[0.01...,"[nan, ...",0,The Ba...
3,Archit...,What i...,381,a Mari...,The Gr...,Grotto...,[Archi...,[The G...,4,"[[0.0,...",[[0.07...,[nan],0,The Gr...
4,Archit...,What s...,92,a gold...,Univer...,top Ma...,[Archi...,[Unive...,1,"[[0.0,...",[[0.16...,"[nan, ...",0,Univer...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,In 191...,In 191...,8,Father...,The Un...,presid...,[In 19...,[The U...,0,"[[0.0,...",[[0.03...,"[nan, ...",0,The Un...
96,In 191...,Over h...,66,three ...,The Un...,years ...,[In 19...,[The U...,0,"[[0.0,...",[[0.05...,"[nan, ...",0,The Un...
97,In 191...,Those ...,430,Harvar...,St. Jo...,Jesuit...,[In 19...,[St. J...,2,"[[0.0,...",[[0.02...,"[nan, ...",0,St. Jo...
98,One of...,The No...,117,Knute ...,This i...,Notre ...,[One o...,[This ...,4,"[[0.0,...",[[0.04...,"[nan, ...",0,This i...


In [89]:
pd.set_option('display.max_colwidth', 10)

In [88]:
pd.set_option('display.max_rows', 10)

In [98]:
predicted[['question','pred_sen_cos','text']]

,question,pred_sen_cos,text
0,To who...,Our La...,Saint ...
1,What i...,Univer...,a copp...
2,The Ba...,The Ba...,the Ma...
3,What i...,The Gr...,a Mari...
4,What s...,Univer...,a gold...
...,...,...,...
95,In 191...,The Un...,Father...
96,Over h...,The Un...,three ...
97,Those ...,St. Jo...,Harvar...
98,The No...,This i...,Knute ...


In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
print_full(predicted.iloc[10,6])